In [11]:
import sys
import math, random

import rospy
import tf

from moveit_msgs.srv import GetPositionIK
from moveit_msgs.msg import PositionIKRequest, RobotState, DisplayRobotState

from visualization_msgs.msg import Marker
from geometry_msgs.msg import PoseStamped


import moveit_commander


In [12]:
rospy.wait_for_service('compute_ik')
compute_ik = rospy.ServiceProxy('compute_ik', GetPositionIK)

pub_markers = rospy.Publisher('visualization_marker', Marker, queue_size=10)
pub_ik_target = rospy.Publisher('ik_target', PoseStamped, queue_size=10)

rospy.init_node("sample_ik_reachable")

tl = tf.TransformListener()

In [13]:
robot = moveit_commander.RobotCommander()

In [14]:
robot.get_link_names()

['world',
 'base_link',
 'base',
 'shoulder_link',
 'upper_arm_link',
 'forearm_link',
 'wrist_1_link',
 'wrist_2_link',
 'wrist_3_link',
 'ee_link',
 'tool0']

In [3]:
link = robot.get_link("ee_link")
link.pose()
group = moveit_commander.MoveGroupCommander("manipulator")
print "============ End effector: %s" % group.get_end_effector_link()


NameError: name 'robot' is not defined

In [2]:
group.get_current_pose()

NameError: name 'group' is not defined

In [17]:
def get_ik(target, initial_state, group = "manipulator"):
    """
    :param target:  a PoseStamped give the desired position of the endeffector.
    """
    service_request = PositionIKRequest()
    service_request.group_name = group
    service_request.robot_state = initial_state
    service_request.ik_link_name = "ee_link"
    service_request.pose_stamped = target
    service_request.timeout.secs= 0.1
    service_request.avoid_collisions = False

    try:
        resp = compute_ik(ik_request = service_request)
        return resp
    except rospy.ServiceException, e:
        print "Service call failed: %s"%e


In [1]:

x = -0.0377490661104
y = -0.516911386609
z = 0.408559164639

target.pose.position.x = x
target.pose.position.y = y
target.pose.position.z = z

pub_ik_target.publish(target)

print("Checking %s %s %s" %(x,y,z))

res = get_ik(target, robot.get_current_state())

print res.error_code


NameError: name 'target' is not defined

In [30]:
res.solution.joint_state.position

(1.1254808887152379,
 -1.2790473577594499,
 1.0253452988633909,
 0.25370204289607934,
 2.696277216758782,
 -3.14159265359)